In [1]:
import pandas as pd

In [2]:
import numpy as np 
from io import StringIO
from typing import Dict
from sklearn.ensemble import RandomForestRegressor

import pandas as pd

In [3]:
def string_to_pandas(text):
    stringio = StringIO(text[1])
    df = pd.read_csv(stringio).drop('Unnamed: 0', axis=1)
    return get_keys(text[0]), df

def get_keys(x):
    try:
        name = os.path.basename(x)
        return "".join([s for s in name if s.isdigit()])
    except (IndexError, KeyError):
        return '', ''
    
def concat(x):
    train = x[1][0][1]
    train["train"] = 1
    
    val = x[1][1][1]
    val["train"] = 0
    
    train = pd.concat([train, gene_expression_train_bc.value], axis=1)
    val = pd.concat([val, gene_expression_validation_bc.value], axis=1)

    
    return pd.concat([train, val], axis=0, ignore_index=True)    


In [4]:
validation_rdd = sc.wholeTextFiles("gs://conrad-project/validation_prediction/spark_input_data/validation_combat_9_03_18/*", minPartitions=500)
training_rdd = sc.wholeTextFiles("gs://conrad-project/validation_prediction/spark_input_data/train_combat_9_03_18/*", minPartitions=500)

In [5]:
gene_expression_train = spark.read.option("header", True).csv("gs://conrad-project/validation_prediction/spark_input_data/training_set_gene_expression.csv").toPandas()
gene_expression_validation = spark.read.option("header", True).csv("gs://conrad-project/validation_prediction/spark_input_data/validation_set_gene_expression.csv").toPandas()

In [6]:
cols = [ "gene_" + c for c in gene_expression_train.columns.tolist()]
cols[0] = "name"
gene_expression_train.columns = cols
print(len(set(cols)))

cols = [ "gene_" + c for c in gene_expression_validation.columns.tolist()]
cols[0] = "name"
gene_expression_validation.columns = cols
print(len(set(cols)))


4684
4684


In [7]:
gene_expression_train_bc = sc.broadcast(gene_expression_train)
gene_expression_validation_bc = sc.broadcast(gene_expression_validation)

In [8]:
validation_rdd = validation_rdd.map(lambda x: string_to_pandas(x)).keyBy(lambda x: get_keys(x[0]))
training_rdd = training_rdd.map(lambda x: string_to_pandas(x)).keyBy(lambda x: get_keys(x[0]))
joined_rdd = training_rdd.join(validation_rdd, numPartitions=500).map(concat)


In [9]:
test = validation_rdd.take(1)[0]

In [10]:
t = joined_rdd.take(1)[0]

In [14]:
def forecast(data, target, features, clf):

    data = data.apply(pd.to_numeric, errors="ignore")
    data[target] = np.log(data[target])
    if data.shape[0] < 10:
        return []

    data = data.reset_index(drop=True)
    data_train = data[data["train"] == 1]
    data_test  =  data[(data["train"] == 0)]

    data_train = data_train.replace([np.inf, -np.inf], np.nan)
    data_test = data_test.replace([np.inf, -np.inf], np.nan)
    data_train = data_train.dropna(subset=features+[target])
    data_test = data_test.dropna(subset=features+[target])
    
    
    importances = []
    if data_test.shape[0] > 0:
        num_training = data_train.shape[0]
        df = pd.get_dummies(pd.concat([data_train, data_test])[features])

        X_train = df.iloc[:num_training, :]
        X_test = df.iloc[num_training:, :]

        y_train = data_train[target].values.ravel()
        clf.fit(X_train, y_train)
        prediction = clf.predict(X_test)
        data_test["prediction"] = np.exp(prediction)
        importances.append(clf.feature_importances_)
        for r in range (0, 100):
            np.random.shuffle(y_train)
            clf.fit(X_train, y_train)
            prediction = clf.predict(X_test)
            importances.append(clf.feature_importances_)
            data_test["prediction_{}".format(r)] = np.exp(prediction)
        return [data_test.filter(regex="prediction*"), pd.DataFrame(importances)]
    return []

In [15]:
clf = RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1,
                                min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, bootstrap=True,
                                oob_score=True, n_jobs=-1, random_state=None,
                                verbose=0, warm_start=False)


In [16]:
test = joined_rdd.take(1)[0]
features = test.drop(['BMI.catg', 'BMI.norm', 'name'], axis=1).columns.tolist()

In [18]:
forecast_rdd = joined_rdd.map(lambda x: forecast(x, "BMI.norm", features, clf))

In [ ]:
predictions = forecast_rdd.collect()

In [39]:
results_pred = []
importances = []
for i, p in enumerate(predictions):
    pred = p[0]
    impor = p[1]
    pred["imputation_set"] = i
    impor["imputation_set"] = i
    importances.append(impor)
    results_pred.append(pred)
    

In [41]:
results_pred = pd.concat(results_pred)

In [42]:
importances = pd.concat(importances)

In [44]:
results_pred.to_csv("/home/tomi/data/results_pred.scv", index=False)

In [45]:
importances.to_csv("/home/tomi/data/importances.scv", index=False)